In [1]:
# importing the module
import json
import re
import pandas as pd
import numpy as np
from IPython.display import display
from utils import *
from file_reader import *
from df_utils import *

In [38]:
def get_all_project_id(file_name):
    df = pd.read_csv(file_name, index_col=0)
    #display(df.index)
    return df.index
    

#project_id_list = get_all_project_id('project_df_page_0_to_276_cut_10_10.csv')


def get_lib_list(path,id_list,output_df):
    lib_list=[]
    df_header=['projectID','libID','rating']
    df = pd.DataFrame(columns=df_header)
    for i in id_list:
        libs = read_file(path,i)
        for j in libs:
            df2 = pd.DataFrame({'userID': [i],
                            'itemID': [j],
                            'rating': [1]})
            df = df.append(df2, ignore_index = True)
        lib_list = lib_list+libs
    #df.to_csv(output_df, index = False)
    #display(df)
    lib_set = set(lib_list)
    return lib_set,df


def create_lib_surprise_df(path,id_list, output_df):
    libs,temp_df = get_lib_list(path,id_list,output_df)
    display(len(temp_df))
    list_of_zeroes = [0]*(len(libs)*len(id_list))
    index = pd.MultiIndex.from_product([id_list, libs], names = ["userID", "itemID"])
    df = pd.DataFrame(index = index)
    df['rating'] = list_of_zeroes
    for i in range(0, len(temp_df)):
        #print (temp_df.loc[i,'userID'])
        df.loc[temp_df.loc[i,'userID'],temp_df.loc[i,'itemID']]=1
    df.to_csv(output_df)
    display(df)

create_lib_surprise_df('C:\\progetti\\low-code-ard-proj\\data-scraper\\code-snippets\\', get_all_project_id('project_df_page_0_to_276_cut_0_0.csv'),output_df='complete_project_lib_df.csv')
#get_lib_list('C:\\progetti\\low-code-ard-proj\\data-scraper\\code-snippets\\',project_id_list)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
"""
this function take as input a df with all information of the project (df1)
the (df2) with the relation between project and libraries
and return a new dataframe with the relation between components and libraries
"""



def add_libraries_to_df(df1,df2,output_file):
    df1=df1.set_index(['project_id'])  
    df2= df2[df2['rating'] != 0]
    df1['libraries']=''
    srs=df2.groupby(['userID'])['itemID'].apply(list)
    for i in srs.keys():
        df1.at[i,'libraries'] =srs [i]
    df1= df1[df1['libraries'] !='' ]
    df1 =create_df_from_2_df_col(df1,output_file='out.csv',col_1='components', col_2 ='libraries',row_cut=0,col_cut=0)
    df1.to_csv(output_file)
    return df1
    

df_with_lib=add_libraries_to_df(df1 = json_df_to_pd_df('all_project_detail_final_201022.txt'),df2 = pd.read_csv('complete_project_lib_df.csv'),output_file='complete_df_comp_lib.csv')   


,"#include ""Chronodot.h""",#include <PS3BT.h>,#include <CayenneYun.h>,#include <Temporizador.h>,"#include ""Phpoc.h""","#include ""fsl_lpuart.h""","#include ""TFT_22_ILI9225.h""","#include ""BasicStepperDriver.h""","#include ""ST7565.h""","#include ""Temporizador.h""",...,#include <FastLED_NeoMatrix.h>,#include <Arduino_SensorKit_BMP280.h>,#include <epd2in13b.h>,"#include ""IOExpander.h""","#include ""BLEHM10.h""","#include ""Color.h""","#include ""air_quality_level.h""","#include ""avr/dtostrf.h""","#include ""SHT31.h""",#include <windows.h>
OOTDTY 2 Channel SSR Solid State Relay High Lever Trigger 5A 5v12v For Uno R3 APR11_10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
\t2WD Robot Car Chassis for Arduino,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Delkin Industrial SLC microSD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Panasonic 680 uF capacitor,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SparkFun SO5NF STD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YwRobot Arduino LCM1602 IIC V1 conventer,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTP223 Capacitive Touch Sensor,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Plastic Enclosure (clear lid),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
D-duino32,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Robot Base Frame / Chassis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def cut_off_by_non_zero_element(df, row_cut,col_cut):
    for c in df.columns:
        if len(df[df[c]>0])<col_cut+1:
            df=df.drop([c], axis=1)
    #return the series of non zero values per row
    serie=df.apply(np.flatnonzero, axis=1)
    for t in df.index:
        if(len(serie[t])<row_cut+1):
            df=df.drop([t], axis=0)
    return df


cutted_df = cut_off_by_non_zero_element(df_with_lib, 1,5)


In [24]:
def cut_and_save_comp_lib_df(df1, row_cut=0, col_cut=0, output_file= 'surprise_df_comp_lib.csv'):
    cutted_df = cut_off_by_non_zero_element(df1, row_cut,col_cut)
    cutted_df.to_csv(output_file)
    return cutted_df

In [ ]:
   
cut_and_save_comp_lib_df(pd.read_csv('complete_project_lib_df.csv'), row_cut=2, col_cut=10, output_file= 'df_comp_lib_cut_2_10.csv')

,#include <PS3BT.h>,#include <CayenneYun.h>,"#include ""Phpoc.h""","#include ""fsl_lpuart.h""","#include ""BasicStepperDriver.h""","#include ""ST7565.h""",#include <TinyGPS++.h>,#include <TH02_dev.h>,"#include ""debug.h""",#include <DS3232RTC.h>,...,#include <Adafruit_MotorShield.h>,"#include ""gas.c""","#include ""font.h""","#include ""ship_pins.h""","#include ""nRF24L01.h""",#include <LiquidCrystal_PCF8574.h>,#include <Adafruit_NeoPixel.h>,#include <Arduino_HTS221.h>,"#include ""BLEHM10.h""","#include ""air_quality_level.h"""
Delkin Industrial SLC microSD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Panasonic 680 uF capacitor,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
D-duino32,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Robot Base Frame / Chassis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Seeed Studio 2.8 TFT Touch Shield V2.0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Potentiometer (10K),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Battery Holder AA x 8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
Photo interrupter,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RFID Writer,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
cut_and_save_comp_lib_df(pd.read_csv('complete_project_lib_df.csv'), row_cut=5, col_cut=10, output_file= 'df_comp_lib_cut_5_10.csv')

NameError: name 'df_with_lib' is not defined

In [ ]:
cut_and_save_comp_lib_df(pd.read_csv('complete_project_lib_df.csv'), row_cut=5, col_cut=20, output_file= 'df_comp_lib_cut_5_20.csv')

In [ ]:
cut_and_save_comp_lib_df(pd.read_csv('complete_project_lib_df.csv'), row_cut=10, col_cut=10, output_file= 'df_comp_lib_cut_10_10.csv')

,#include <PS3BT.h>,#include <CayenneYun.h>,"#include ""Phpoc.h""","#include ""fsl_lpuart.h""","#include ""BasicStepperDriver.h""","#include ""ST7565.h""",#include <TinyGPS++.h>,#include <TH02_dev.h>,"#include ""debug.h""",#include <DS3232RTC.h>,...,#include <Adafruit_MotorShield.h>,"#include ""gas.c""","#include ""font.h""","#include ""ship_pins.h""","#include ""nRF24L01.h""",#include <LiquidCrystal_PCF8574.h>,#include <Adafruit_NeoPixel.h>,#include <Arduino_HTS221.h>,"#include ""BLEHM10.h""","#include ""air_quality_level.h"""
Seeed Studio 2.8 TFT Touch Shield V2.0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
TFT Touchscreen 320x240,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LED Matrix I2C interface,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Emergency Stop Button,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Espressif Wemos D1 Mini,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GY-BMP280 breakout board,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4xAA battery holder,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,2,0,0,0
AQY 212 GH,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
IC & Component Socket 8 Contacts,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_plot = pd.read_csv('df_comp_lib_cut_0_0.csv')


In [ ]:



def df_to_surprise_df(input_file='json_test.txt',output_file='out.csv',index_names = ["index_1", "index_2"],column_name='rating'):
    df=pd.read_csv(input_file, index_col=0)
    index_1 =df.index
    index_2=df.columns
    #print(len(index_2))
    zeroes = [0]*(len(index_1)*len(index_2))
    index = pd.MultiIndex.from_product([index_1, index_2], names = index_names)
    df2= pd.DataFrame(index = index)
    df2['rating']=zeroes
    for i in df.index:
        for j in df.columns:
            if df.loc[i,j]!=0:
                df2.loc[i,j]=df.loc[i,j]
    df2.to_csv(output_file)
    return df2


surp=df_to_surprise_df(input_file='df_comp_lib_cut_2_10.csv',output_file='surprise_df_comp_lib_cut_2_10.csv', index_names = ["componentID", "libID"],column_name='rating')
display(surp)

rating
componentID                   libID                                     
Delkin Industrial SLC microSD #include <PS3BT.h>                       0
                              #include <CayenneYun.h>                  0
                              #include "Phpoc.h"                       0
                              #include "fsl_lpuart.h"                  0
                              #include "BasicStepperDriver.h"          0
...                                                                  ...
STMicroelectronics VL53L0X    #include <LiquidCrystal_PCF8574.h>       0
                              #include <Adafruit_NeoPixel.h>           0
                              #include <Arduino_HTS221.h>              0
                              #include "BLEHM10.h"                     0
                              #include "air_quality_level.h"           0

[3001954 rows x 1 columns]

In [25]:
surp=df_to_surprise_df(input_file='df_comp_lib_cut_10_10.csv',output_file='surprise_df_comp_lib_cut_10_10.csv', index_names = ["componentID", "libID"],column_name='rating')

In [ ]:
surp=df_to_surprise_df(input_file='original_df_plus_lib_test.csv',output_file='surprise_comp_lib_test.csv', index_names = ["componentID", "libID"],column_name='rating')

In [2]:
predict_and_validate(surprise_df='surprise_df_comp_lib_cut_2_10.csv',sim_options={
            'name': 'msd',
            'user_based': True,
            'min_support':1,
            'shrinkage':0
            },threshold = 0.2,k=10,fold_split=10)

Computing the msd similarity matrix...
Done computing similarity matrix.
PRECISION: 0.25406768814525904
RECALL: 0.2641653506170672
RMSE: 0.5012
Computing the msd similarity matrix...
Done computing similarity matrix.
PRECISION: 0.25601222372082283
RECALL: 0.26417890513780884
RMSE: 0.4845
Computing the msd similarity matrix...
Done computing similarity matrix.
PRECISION: 0.2601090738243072
RECALL: 0.2652420276332424
RMSE: 0.6077
Computing the msd similarity matrix...
Done computing similarity matrix.
PRECISION: 0.2481670494173684
RECALL: 0.2589710995701681
RMSE: 0.6278
Computing the msd similarity matrix...
Done computing similarity matrix.
PRECISION: 0.2576026642689056
RECALL: 0.2661928825202017
RMSE: 0.4682
Computing the msd similarity matrix...
Done computing similarity matrix.
PRECISION: 0.25769835194433294
RECALL: 0.2663792368260172
RMSE: 0.4581
Computing the msd similarity matrix...
Done computing similarity matrix.
PRECISION: 0.2656708060446258
RECALL: 0.2703388401247222
RMSE: 0.

In [3]:
predictions=get_all_prediction('surprise_df_comp_lib_cut_2_10.csv')
precision,recall,accuracy,fprate = get_precision_recall_accuracy_fprate(predictions, 0.5)

print('-----compoment/libraries prediction statistics------')
print('precision:',precision,'recall:',recall,'accuracy',accuracy,'false positive rate:',fprate)

Computing the msd similarity matrix...
Done computing similarity matrix.
-----compoment/libraries prediction statistics------
precision: 0.8031693077564637 recall: 0.1111624148678287 accuracy 0.9894252512102464 false positive rate: 0.0003181433370539927


In [4]:
display (predictions)

[Prediction(uid='Waterproof Metal Push Button Switch LED Light ', iid='#include <WebSocketsClient.h>', r_ui=0.0, est=0, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='microSD card', iid='#include <TEA5767.h>', r_ui=0.0, est=0, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='ws2812b LEDs', iid='#include "Ultrasonic.h"', r_ui=0.0, est=0, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='LED Dot Laser (generic)', iid='#include "button.h"', r_ui=0.0, est=0, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='nrf24l01', iid='#include "UBeeZ_Prefs.h"', r_ui=0.0, est=0, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='Socket (one way)', iid='#include <TinyGPS++.h>', r_ui=0.0, est=0.02500138896605367, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='M3x15 metal hex spacer standoffs', iid='#include "pin_mux.h"', r_ui=0.0, est=0, details={'actual_k': 40, 'was_impossible': False}

In [ ]:
from matplotlib.pyplot import get
from surprise import Dataset,accuracy, NormalPredictor, Reader,SVD, KNNWithMeans,KNNBasic, KNNWithZScore
from surprise.model_selection import cross_validate, train_test_split,KFold
import heapq
from collections import defaultdict
from operator import itemgetter


def precision_recall_at_k(predictions, k=10, threshold=0.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for  uid,_, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))
    #print(user_est_true)
    #display(user_est_true)

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        #print('relevant: ',n_rel)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        #print('recommended: ',n_rec_k)

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls






df = pd.read_csv('test_lib_df.csv')
 
#display(df)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0, 1))
#print(df.columns)
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[df.columns], reader)
algo = KNNWithMeans(sim_options={
        'name': 'msd',
        'user_based': True,
        })
        #'min_support':5,
        #'shrinkage':0

# Retrieve the trainset.
#trainset, testset = train_test_split(data, test_size=0.25)


# define a cross-validation iterator
kf = KFold(n_splits=2)


for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=0.1)
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))
    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)


# Train the algorithm on the trainset, and predict ratings for the testset
#algo.fit(trainset)
#predictions = algo.test(testset)
# We can now use this dataset as we please, e.g. calling cross_validate
#cross_validate(NormalPredictor(), data, cv=10)
#cross_validate(algo, data, measures=["RMSE", "MAE"], cv=10, verbose=True)



Computing the msd similarity matrix...
Done computing similarity matrix.
0.11871794871794872
0.16384615384615384
RMSE: 0.1615
Computing the msd similarity matrix...
Done computing similarity matrix.
0.18794871794871792
0.16813186813186812
RMSE: 0.1646


In [5]:
display (predictions)

[Prediction(uid='Waterproof Metal Push Button Switch LED Light ', iid='#include <WebSocketsClient.h>', r_ui=0.0, est=0, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='microSD card', iid='#include <TEA5767.h>', r_ui=0.0, est=0, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='ws2812b LEDs', iid='#include "Ultrasonic.h"', r_ui=0.0, est=0, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='LED Dot Laser (generic)', iid='#include "button.h"', r_ui=0.0, est=0, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='nrf24l01', iid='#include "UBeeZ_Prefs.h"', r_ui=0.0, est=0, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='Socket (one way)', iid='#include <TinyGPS++.h>', r_ui=0.0, est=0.02500138896605367, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='M3x15 metal hex spacer standoffs', iid='#include "pin_mux.h"', r_ui=0.0, est=0, details={'actual_k': 40, 'was_impossible': False}

In [18]:


def get_success_rate(predictions, n=10, threshold = 0.3):
    # First map the predictions to each user.
    
    top_n =get_top_n(predictions=predictions, n=n)
    total = len(top_n)
    n_success = 0
    for i in top_n:
        
        partial_success=0
        for j in top_n[i]:
            print(j)
            _,est,true_r = j
            if(true_r > threshold and est > threshold):
                partial_success=1
        n_success+=partial_success
            
        print('--------')
    return n_success/total

success_rate = get_success_rate(predictions, n=5, threshold=0.2)

('#include <Ultrasonic.h>', 0.3753914566146439, 1.0)
('#include <Servo.h>', 0.14989767184842598, 0.0)
('#include <Stepper.h>', 0.07496387272751101, 0.0)
('#include <EEPROM.h>', 0.07496315733492878, 0.0)
('#include "DHT.h"', 0.04994515257718706, 0.0)
--------
('#include <LiquidCrystal_I2C.h>', 0.3250238097268823, 0.0)
('#include <Adafruit_VS1053.h>', 0.10000519539668329, 0.0)
('#include <Servo.h>', 0.10000220035938645, 0.0)
('#include <OneWire.h>', 0.04996926959654305, 0.0)
('#include <TinyGPS++.h>', 0.04996776165687371, 0.0)
--------
('#include <EEPROM.h>', 0.27493931071928046, 0.0)
('#include <TinyGPS++.h>', 0.075045994059758, 0.0)
('#include "FastLED.h"', 0.07493601605621672, 0.0)
('#include "SSD1306AsciiAvrI2c.h"', 0.05003261096062532, 0.0)
('#include <Ultrasonic.h>', 0.050025874474936394, 0.0)
--------
('#include <SD.h>', 0.12497888829424315, 0.0)
('#include <TimeLib.h>', 0.12493509888822721, 0.0)
('#include "Adafruit_TCS34725.h"', 0.07506252176382805, 0.0)
('#include <FastLED.h>',

In [19]:
print (success_rate)

0.5011482521051288


In [27]:
predictions=get_all_prediction('surprise_df_comp_lib_cut_10_10.csv')
precision,recall,accuracy,fprate = get_precision_recall_accuracy_fprate(predictions, 0.5)

print('-----compoment/libraries prediction statistics------')
print('precision:',precision,'recall:',recall,'accuracy',accuracy,'false positive rate:',fprate)



Computing the msd similarity matrix...
Done computing similarity matrix.
-----compoment/libraries prediction statistics------
precision: 0.8299776286353467 recall: 0.17037887485648678 accuracy 0.9656283664116563 false positive rate: 0.0014451004439880969


In [28]:
success_rate_10_10 = get_success_rate(predictions, n=5, threshold=0.2)

print (success_rate_10_10)

('#include <Wire.h>', 0.9751724153619816, 1.0)
('#include <SigFox.h>', 0.2500254503322204, 0.0)
('#include <Helium.h>', 0.22466224253099357, 0.0)
('#include <ArduinoLowPower.h>', 0.19997806355238681, 0.0)
('#include <Arduino.h>', 0.19981492754787744, 0.0)
--------
('#include <Adafruit_SSD1306.h>', 0.37414663786587865, 0.0)
('#include "tensorflow/lite/micro/all_ops_resolver.h"', 0.17474522876058374, 0.0)
('#include <ArduinoLowPower.h>', 0.17449084906993917, 0.0)
('#include <Adafruit_BMP085.h>', 0.1498174368694157, 0.0)
('#include <Servo.h>', 0.14963992858028347, 0.0)
--------
('#include <SPI.h>', 0.49602755053184716, 0.0)
('#include <IRremote.h>', 0.2981323365205643, 0.0)
('#include <NanoBLEFlashPrefs.h>', 0.28165653188416595, 1.0)
('#include "UBeez_Bluetooth.h"', 0.2816508612209676, 1.0)
('#include <SD.h>', 0.24804501977582635, 0.0)
--------
('#include <Wire.h>', 1.773626108188336, 6.0)
('#include <Fonts/FreeMonoBold24pt7b.h>', 0.20069212258010888, 1.0)
('#include <stdio.h>', 0.1746094

In [ ]:
def print_predictions(real, lower_bound):
    count = 0
    for i in range(0,len(predictions)):
        if(predictions[i][2]==real and predictions[i][3]>lower_bound):
            count= count +1
            print(count, predictions[i][2],predictions[i][3])
       
print_predictions(1, 0.1)

In [ ]:
import random

from surprise import accuracy, Dataset, SVD
from surprise.model_selection import GridSearchCV


df = pd.read_csv('surprise_libraries_df.csv')
 
reader = Reader(rating_scale=(0, 1))

data = Dataset.load_from_df(df[df.columns], reader)

raw_ratings = data.raw_ratings

# shuffle ratings if you want
random.shuffle(raw_ratings)

# A = 90% of the data, B = 10% of the data
threshold = int(0.9 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]

data.raw_ratings = A_raw_ratings  # data is now the set A

# Select your best algo with grid search.
print("Grid Search...")
param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005]}
grid_search = GridSearchCV(SVD, param_grid, measures=["rmse"], cv=3)
grid_search.fit(data)

algo = grid_search.best_estimator["rmse"]

# retrain on the whole set A
trainset = data.build_full_trainset()
algo.fit(trainset)

# Compute biased accuracy on A
predictions = algo.test(trainset.build_testset())
print("Biased accuracy on A,", end="   ")
accuracy.rmse(predictions)

# Compute unbiased accuracy on B
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
predictions = algo.test(testset)
print("Unbiased accuracy on B,", end=" ")
accuracy.rmse(predictions)